La información contenida en objetos pandas, puede ser combinada de diferentes maneras:

- pandas.merge: conecta filas de un dataframe basándose en uno o más valores llave. Operación similar a los joins de SQL.
- pandas.concat: concatena matrices, puede ser con numpy o pandas.
- combine_first: empalma datos para llenar vacíos.

# Merge o Join

### Inner Join

Como dijimos, combinan filas usando una o más llaves. Empezamos con un ejemplo:

In [2]:
import pandas as pd
import numpy as np

In [4]:
df1= pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                  "data1": range(7)})

print(df1)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6


In [5]:
df2= pd.DataFrame({"key": ["a", "b", "d"],
                  "data2": range(3)})

print(df2)

  key  data2
0   a      0
1   b      1
2   d      2


Este es el típico caso de un busvarv. Mientras en el primer dataset tenemos varios valores repetidos en la columna "key", en el segundo dataset, no tenemos valores repetidos. Lo que se conoce como "Many to one":

In [12]:
df3= pd.merge(df1, df2)  

print(df3)

  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0


Notese que en el caso anterior no indicamos la columna llave para hacer la unión. Es una buena práctica hacerlo de la siguiente manera: 

In [7]:
df4= pd.merge(df1, df2, on= "key")

print(df4)

  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0


Hagamos de cuenta que las columnas llave tienen diferente nombre:

In [8]:
df5= pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                   "data1": range(7)})

print(df5)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6


In [9]:
df6= pd.DataFrame({"rkey": ["a", "b", "d"],
                   "data2": range(3)})

print(df6)

  rkey  data2
0    a      0
1    b      1
2    d      2


In [11]:
df7= pd.merge(df5, df6, left_on= "lkey", right_on= "rkey")

print(df7)

  lkey  data1 rkey  data2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0


Como se puede observar, el resultado es un inner join, se puede hacer otros tipos de join de la sgte manera:

- left
- right
- outer (es el full join)


In [13]:
df8= pd.merge(df1, df2, how= "left")

print(df8)

  key  data1  data2
0   b      0    1.0
1   b      1    1.0
2   a      2    0.0
3   c      3    NaN
4   a      4    0.0
5   a      5    0.0
6   b      6    1.0


## Left Join

In [14]:
df9= pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})

print(df9)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5


In [15]:
df10= pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": range(5)})

print(df10)

  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [16]:
df11= pd.merge(df9, df10, on= "key", how= "left")

print(df11)

   key  data1  data2
0    b      0    1.0
1    b      0    3.0
2    b      1    1.0
3    b      1    3.0
4    a      2    0.0
5    a      2    2.0
6    c      3    NaN
7    a      4    0.0
8    a      4    2.0
9    b      5    1.0
10   b      5    3.0


### Merge por Múltiples Variables key

In [18]:
left= pd.DataFrame({"key1": ["foo", "foo", "bar"],
                    "key2": ["one", "two", "one"],
                    "lval": [1, 2, 3]})

print(left)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3


In [21]:
right= pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                    "key2": ["one", "one", "one", "two"],
                    "rval": [4, 5, 6, 7]})
                    
print(right)

  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [22]:
ready= pd.merge(left, right, on= ["key1", "key2"], how= "left")

print(ready)

  key1 key2  lval  rval
0  foo  one     1   4.0
1  foo  one     1   5.0
2  foo  two     2   NaN
3  bar  one     3   6.0


En los merge, los índices son descartados

### Columnas que coinciden en nombre

In [25]:
ready1= pd.merge(left, right, on= "key1", how= "left")

print(ready1)

  key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7


Por defecto, a los efectos de que no se repitan los nombres, python coloca la terminación "_x" y "_y". No obstante, nosotros podemos indicar los sufijos que queramos de la sgte manera:

In [27]:
ready2= pd.merge(left, right, on= "key1", how= "left", suffixes= ("_left", "_right"))

print(ready2)

  key1 key2_left  lval key2_right  rval
0  foo       one     1        one     4
1  foo       one     1        one     5
2  foo       two     2        one     4
3  foo       two     2        one     5
4  bar       one     3        one     6
5  bar       one     3        two     7


# Concatenar a lo largo de una fila

### Usando Numpy

Básicamente se trata de concatenar matrices, bajo numpy el método es el siguiente:

In [3]:
arr= np.arange(12).reshape((3,4))

print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [6]:
arr1= np.concatenate([arr, arr], axis= 1)

print(arr1)

[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]]


### Usando Pandas

In [33]:
s1= pd.Series([0, 1], index= ["a", "b"])

s2= pd.Series([2, 3, 4], index= ["c", "d", "e"])

s3= pd.Series([5, 6], index= ["f", "g"])

print(s1)
print("--------------------------")
print(s2)
print("--------------------------")
print(s3)

a    0
b    1
dtype: int64
--------------------------
c    2
d    3
e    4
dtype: int64
--------------------------
f    5
g    6
dtype: int64


In [36]:
s4= pd.concat([s1, s2, s3], axis= 0)  #axis= 0 apenda para abajo

print(s4)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64


In [37]:
s5= pd.concat([s1, s2, s3], axis= 1)  #axis= 1 apenda para el costado, siempre que coincidan los índices

print(s5)

     0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0


Trabajando con axis= 1, también podemos dar la indicación de join= "inner", para que sólo aparezcan aquellas que están en ambas tablas

En alguna versión, se podía indicar los índices que queríamos usar. Actualmente no me permite usarla:

In [48]:
s6= pd.concat([s1, s3])

print(s6)

a    0
b    1
f    5
g    6
dtype: int64


In [49]:
#s7= pd.concat([s1, s6], axis= 1, join_axes=[["a", "c", "b", "e"]])

#print(s7)

La lógica es la misma al usar DataFrames:

In [2]:
df1= pd.DataFrame(np.random.randn(3, 4), columns= ["a", "b", "c", "d"])

print(df1)

          a         b         c         d
0  0.260531  0.583142 -0.407826  0.546556
1  0.532524  0.193869  0.339116 -0.263985
2  1.212656  0.378886 -1.522684  0.175709


In [53]:
df2= pd.DataFrame(np.random.randn(2, 3), columns= ["b", "d", "a"])

print(df2)

          b         d         a
0  2.272472  0.779073 -0.273988
1  0.280470 -0.064792 -0.685696


In [54]:
df3= pd.concat([df1, df2])

print(df3)

          a         b         c         d
0  0.974672  0.793093 -2.021308  0.135742
1  0.625530  1.164840 -0.918084  1.297364
2 -1.665539  0.379139 -0.237256  0.702509
0 -0.273988  2.272472       NaN  0.779073
1 -0.685696  0.280470       NaN -0.064792


Si el index no tiene ninguna utilidad, podemos hacer lo siguiente:

In [55]:
df4= pd.concat([df1, df2], ignore_index= True)

print(df4)

          a         b         c         d
0  0.974672  0.793093 -2.021308  0.135742
1  0.625530  1.164840 -0.918084  1.297364
2 -1.665539  0.379139 -0.237256  0.702509
3 -0.273988  2.272472       NaN  0.779073
4 -0.685696  0.280470       NaN -0.064792


# Combine con overlap

No es nada que no se pueda hacer con los otros métodos:

In [2]:
df1= pd.DataFrame({"a": [1., np.nan, 5, np.nan],
                   "b": [np.nan, 2., np.nan, 6.],
                   "c": range(2, 18, 4)})

print(df1)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14


In [4]:
df2= pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                   "b": [np.nan, 3., 4., 6., 8.]})

print(df2)

     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [5]:
df3= df1.combine_first(df2)

print(df3)

     a    b     c
0  1.0  NaN   2.0
1  4.0  2.0   6.0
2  5.0  4.0  10.0
3  3.0  6.0  14.0
4  7.0  8.0   NaN


Básicamente, le damos la indicación de que llene uno con los datos de otro, en aquellos lugares donde hay valores vacíos o nan.